In [1]:
%matplotlib inline  

In [2]:
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np


Using TensorFlow backend.


In [3]:
import os

data_dir = './data'
base_train_dir = os.path.join(data_dir, 'train')
base_test_dir = os.path.join(data_dir, 'test')

In [4]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), input_shape=(60, 60, 3), use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu')) 
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(32, (3, 3), use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(64, (3, 3), use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(128, (3, 3), use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
#model.add(layers.Conv2D(64, (3, 3), use_bias=False))
#model.add(layers.BatchNormalization())
#model.add(layers.Activation('relu'))
#model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dropout(0.4))

model.add(layers.Dense(512, use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 58, 16)        432       
_________________________________________________________________
batch_normalization_1 (Batch (None, 58, 58, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 58, 58, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 29, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        4608      
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 32)       

In [5]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])

In [6]:
from keras.preprocessing.image import ImageDataGenerator

# VACC 9441 DO 0.2 0.2
#train_datagen = ImageDataGenerator(rescale=1./255,
#                                   rotation_range=20,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True,
#                                   fill_mode='nearest'
#                                  )

#9495 DO 0.2 0.2 mit horizontal
#9808 DO 0.2 0.2 ohne horizontal
#train_datagen = ImageDataGenerator(rescale=1./255,
#                                   rotation_range=5,
#                                   width_shift_range=0.1,
#                                   height_shift_range=0.1,
#                                   shear_range=0.1,
#                                   zoom_range=0.1,
#                                   #horizontal_flip=True,
#                                   fill_mode='nearest'
#                                  )

#9844 DO 0.2 0.2
#train_datagen = ImageDataGenerator(rescale=1./255,
#                                   rotation_range=10,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   fill_mode='nearest'
#                                  )

#9738 0.2 0.2
#9561 0 0
#9854 0.4 0.4
#9727 0.5 0.5
#9745 0.3 0.3
#9802 0.5 0.3
#9809 0.3 0.5
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest'
                                  )


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_train_dir,
        target_size=(60, 60),
        batch_size=128,
        class_mode='binary'
        )

validation_generator = test_datagen.flow_from_directory(
        base_test_dir,
        target_size=(60, 60),
        batch_size=128,
        class_mode='binary'
        )

Found 17016 images belonging to 2 classes.
Found 10991 images belonging to 2 classes.


In [7]:
from keras. callbacks import EarlyStopping

es = EarlyStopping(monitor='val_acc', patience=5+5, restore_best_weights=True)

history = model.fit_generator(
                              train_generator,
                              steps_per_epoch=142, #39209 - find . -name "*.ppm" | wc -l
                              epochs=100,
                              validation_data=validation_generator,
                              validation_steps=86, #12630 - find . -name "*.ppm" | wc -l
                              callbacks=[es], 
                             )

Epoch 1/100
142/142 [==============================] - 23s 161ms/step - loss: 0.0380 - acc: 0.9865 - val_loss: 0.0204 - val_acc: 0.9955
Epoch 2/100
142/142 [==============================] - 20s 143ms/step - loss: 0.0250 - acc: 0.9925 - val_loss: 0.0321 - val_acc: 0.9903
Epoch 3/100
142/142 [==============================] - 20s 141ms/step - loss: 0.0153 - acc: 0.9950 - val_loss: 0.0643 - val_acc: 0.9727
Epoch 4/100
142/142 [==============================] - 20s 143ms/step - loss: 0.0188 - acc: 0.9938 - val_loss: 0.0385 - val_acc: 0.9861
Epoch 5/100
142/142 [==============================] - 20s 142ms/step - loss: 0.0138 - acc: 0.9959 - val_loss: 0.0500 - val_acc: 0.9846
Epoch 6/100
142/142 [==============================] - 20s 142ms/step - loss: 0.0170 - acc: 0.9944 - val_loss: 0.0084 - val_acc: 0.9979
Epoch 7/100
142/142 [==============================] - 20s 142ms/step - loss: 0.0139 - acc: 0.9955 - val_loss: 0.0069 - val_acc: 0.9990
Epoch 8/100
142/142 [===========================

In [8]:
max(history.history['val_acc'])

0.9992721317441543